In [0]:
import keras
import cv2
import numpy as np
import tensorflow as tf
from keras.datasets import mnist
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from tqdm import tqdm

In [0]:
batch_size = 64
epochs = 35
IMAGE_WIDTH = 32
IMAGE_HEIGHT = 32
NUM_CLASSES = 10
TRAIN_SET_SIZE = 2500

In [0]:
np.random.seed(1)
tf.random.set_seed(1)

In [0]:
def preprocess(imgs):
    
    processed = []
    
    for img in tqdm(imgs):
        processed.append(cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT), interpolation = cv2.INTER_AREA))
    
    proccessed = np.array(processed)
    return proccessed.reshape(proccessed.shape[0], IMAGE_WIDTH, IMAGE_HEIGHT, 1)

In [20]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Decrease the training set size
# idx = np.random.choice(len(x_train), size=TRAIN_SET_SIZE, replace=False)
# x_train = x_train[idx]
# y_train = y_train[idx]

x_train = preprocess(x_train)
x_test = preprocess(x_test)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

100%|██████████| 10000/10000 [00:00<00:00, 136242.84it/s]

x_train shape: (60000, 32, 32, 1)
60000 train samples
10000 test samples


In [0]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [0]:
conv_base = VGG16(input_shape = (IMAGE_HEIGHT,IMAGE_WIDTH, 1),
                         include_top = False, weights = None, classes=NUM_CLASSES)

In [0]:
conv_base.trainable = True

In [0]:
model = Sequential()

model.add(conv_base)
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(NUM_CLASSES))
model.add(Activation('softmax'))

In [0]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['categorical_accuracy'])

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [28]:
es = EarlyStopping(monitor='val_categorical_accuracy', mode='max', min_delta=0.01, patience=3)
history = model.fit(x_train,y_train,
              batch_size = batch_size,
              epochs = epochs,
              validation_data = (x_test,y_test),
              shuffle = True,
              callbacks=[es])

Train on 60000 samples, validate on 10000 samples
Epoch 1/35
60000/60000 [==============================] - 29s 489us/step - loss: 0.3264 - categorical_accuracy: 0.8894 - val_loss: 0.0540 - val_categorical_accuracy: 0.9819
Epoch 2/35
60000/60000 [==============================] - 29s 479us/step - loss: 0.0679 - categorical_accuracy: 0.9803 - val_loss: 0.0527 - val_categorical_accuracy: 0.9834
Epoch 3/35
60000/60000 [==============================] - 29s 478us/step - loss: 0.0484 - categorical_accuracy: 0.9864 - val_loss: 0.0365 - val_categorical_accuracy: 0.9886
Epoch 4/35
60000/60000 [==============================] - 29s 479us/step - loss: 0.0361 - categorical_accuracy: 0.9894 - val_loss: 0.0319 - val_categorical_accuracy: 0.9910


In [29]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 236us/step
Test loss: 0.031939150945516304
Test accuracy: 0.9909999966621399
